In [1]:
from keras_facenet import FaceNet
from mtcnn.mtcnn import MTCNN
import cv2, os
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import pickle

embedder = FaceNet()
detector = MTCNN()


Prepare Face Dataset for Embeddings

In [8]:
def extract_face(img_path, required_size=(160, 160)):
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Unable to read image: {img_path}")
        return None

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(img_rgb)
    if not results:
        print(f"⚠️ No face found in: {img_path}")
        return None

    x, y, w, h = results[0]['box']
    x, y = max(x, 0), max(y, 0)
    face = img_rgb[y:y+h, x:x+w]
    face = cv2.resize(face, required_size)
    return face


def load_faces_from_dataset(dataset_path='dataset'):
    embeddings, labels = [], []
    for student_name in os.listdir(dataset_path):
        student_dir = os.path.join(dataset_path, student_name)
        if not os.path.isdir(student_dir):
            continue
        for img_name in os.listdir(student_dir):
            img_path = os.path.join(student_dir, img_name)
            face = extract_face(img_path)
            if face is None:
                continue
            emb = embedder.embeddings([face])[0]
            embeddings.append(emb)
            labels.append(student_name)
    return np.array(embeddings), np.array(labels)


Train Classifier

In [9]:
embeddings, labels = load_faces_from_dataset()

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

classifier = SVC(kernel='linear', probability=True)
classifier.fit(embeddings, labels_encoded)

# Save model and label encoder
with open('face_classifier.pkl', 'wb') as f:
    pickle.dump(classifier, f)

with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print("✅ Trained and saved Face Recognition classifier.")


⚠️ Unable to read image: dataset/Aryan Jha/.DS_Store
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
⚠️ No face found in: dataset/Aryan Jha/17.jpg
⚠️ No face found in: dataset/Aryan Jha/13.jpg
⚠️ No face found in: dataset/Aryan Jha/11.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
⚠️ No face found in: dataset/Priya/7.jpg
⚠️ No face found in: dataset/Priya/6.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━

Save Face Classifier and Label Encoder

In [11]:
import pickle

# Save SVM classifier
with open('face_classifier.pkl', 'wb') as f:
    pickle.dump(classifier, f)

# Save label encoder
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print(" Face classifier and label encoder saved.")


 Face classifier and label encoder saved.
